<a href="https://colab.research.google.com/github/ghitabnr/Orange-detector-model/blob/main/test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import torch
from pathlib import Path
import torch
import os
!git clone https://github.com/ultralytics/yolov5
%cd yolov5
%pip install -r requirements.txt

Cloning into 'yolov5'...
remote: Enumerating objects: 16525, done.
remote: Total 16525 (delta 0), reused 0 (delta 0), pack-reused 16525
Receiving objects: 100% (16525/16525), 15.05 MiB | 17.78 MiB/s, done.
Resolving deltas: 100% (11355/11355), done.
/content/yolov5
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 5.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 749.5/749.5 kB 35.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 7.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 48.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 48.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 77.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 2.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 2.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 8.4 MB/s eta 0:00:0

In [ ]:
model_path = '/content/drive/MyDrive/ColabNotebooks/best.pt'
model = torch.load(model_path, map_location=torch.device('cpu'))

In [ ]:
!python /content/yolov5/detect.py --weights /content/drive/MyDrive/ColabNotebooks/best.pt --img 672 --conf 0.1 --source /content/drive/MyDrive/ColabNotebooks/data_combined/dataset1/OrangeDetection1/valid/images

detect: weights=['/content/drive/MyDrive/ColabNotebooks/best.pt'], source=/content/drive/MyDrive/ColabNotebooks/data_combined/dataset1/OrangeDetection1/valid/images, data=data/coco128.yaml, imgsz=[672, 672], conf_thres=0.1, iou_thres=0.45, max_det=1000, device=, view_img=False, save_txt=False, save_csv=False, save_conf=False, save_crop=False, nosave=False, classes=None, agnostic_nms=False, augment=False, visualize=False, update=False, project=runs/detect, name=exp, exist_ok=False, line_thickness=3, hide_labels=False, hide_conf=False, half=False, dnn=False, vid_stride=1
YOLOv5 🚀 v7.0-295-gac6c4383 Python-3.10.12 torch-2.2.1+cu121 CPU

Fusing layers... 
Model summary: 157 layers, 1760518 parameters, 0 gradients, 4.1 GFLOPs
image 1/90 /content/drive/MyDrive/ColabNotebooks/data_combined/dataset1/OrangeDetection1/valid/images/00020ebf74c4881c_jpg.rf.9f634b40cb032593772c8c9f115467f8.jpg: 672x672 5 oranges, 158.7ms
image 2/90 /content/drive/MyDrive/ColabNotebooks/data_combined/dataset1/Orange

In [ ]:
!python /content/yolov5/detect.py --weights /content/drive/MyDrive/ColabNotebooks/best.pt --img 672 --conf 0.1 --source /content/drive/MyDrive/ColabNotebooks/data_combined/dataset2/cropload/valid/images

detect: weights=['/content/drive/MyDrive/ColabNotebooks/best.pt'], source=/content/drive/MyDrive/ColabNotebooks/data_combined/dataset2/cropload/valid/images, data=data/coco128.yaml, imgsz=[672, 672], conf_thres=0.1, iou_thres=0.45, max_det=1000, device=, view_img=False, save_txt=False, save_csv=False, save_conf=False, save_crop=False, nosave=False, classes=None, agnostic_nms=False, augment=False, visualize=False, update=False, project=runs/detect, name=exp, exist_ok=False, line_thickness=3, hide_labels=False, hide_conf=False, half=False, dnn=False, vid_stride=1
YOLOv5 🚀 v7.0-295-gac6c4383 Python-3.10.12 torch-2.2.1+cu121 CPU

Fusing layers... 
Model summary: 157 layers, 1760518 parameters, 0 gradients, 4.1 GFLOPs
image 1/10 /content/drive/MyDrive/ColabNotebooks/data_combined/dataset2/cropload/valid/images/DSC_0208_JPG_jpg.rf.9999650c3d7c439aff4a5ca18e7b6c05.jpg: 672x672 90 oranges, 171.9ms
image 2/10 /content/drive/MyDrive/ColabNotebooks/data_combined/dataset2/cropload/valid/images/IMG

In [ ]:
!python /content/yolov5/detect.py --weights /content/drive/MyDrive/ColabNotebooks/best.pt --img 672 --conf 0.1 --source /content/drive/MyDrive/ColabNotebooks/data_combined/dataset3/FinalDataset/valid/images

detect: weights=['/content/drive/MyDrive/ColabNotebooks/best.pt'], source=/content/drive/MyDrive/ColabNotebooks/data_combined/dataset3/FinalDataset/valid/images, data=data/coco128.yaml, imgsz=[672, 672], conf_thres=0.1, iou_thres=0.45, max_det=1000, device=, view_img=False, save_txt=False, save_csv=False, save_conf=False, save_crop=False, nosave=False, classes=None, agnostic_nms=False, augment=False, visualize=False, update=False, project=runs/detect, name=exp, exist_ok=False, line_thickness=3, hide_labels=False, hide_conf=False, half=False, dnn=False, vid_stride=1
YOLOv5 🚀 v7.0-295-gac6c4383 Python-3.10.12 torch-2.2.1+cu121 CPU

Fusing layers... 
Model summary: 157 layers, 1760518 parameters, 0 gradients, 4.1 GFLOPs
image 1/15 /content/drive/MyDrive/ColabNotebooks/data_combined/dataset3/FinalDataset/valid/images/IMG_8762_JPG.rf.c16c59c24dd93ed97f1d38b004c10703.jpg: 672x672 122 oranges, 167.5ms
image 2/15 /content/drive/MyDrive/ColabNotebooks/data_combined/dataset3/FinalDataset/valid/i

**VIDEO**

In [ ]:
!python /content/yolov5/detect.py --weights /content/drive/MyDrive/ColabNotebooks/best.pt --img 672 --conf 0.1 --source /content/drive/MyDrive/ColabNotebooks/test/video

DECOUPAGE EN **4**

In [ ]:
import os
from PIL import Image

# Function to crop an image into 4 pieces
def crop_image(image):
    width, height = image.size
    piece_width = width // 2
    piece_height = height // 2
    pieces = []
    for j in range(2):
        for i in range(2):
            box = (i * piece_width, j * piece_height, (i + 1) * piece_width, (j + 1) * piece_height)
            piece = image.crop(box)
            pieces.append(piece)
    return pieces

# Path to the directory containing the input images
input_directory = "/content/drive/MyDrive/ColabNotebooks/test/imagesinput"
output_directory = "/content/drive/MyDrive/ColabNotebooks/test/imagesdecoupees"

# Iterate through all images in the directory
for filename in os.listdir(input_directory):
    if filename.endswith(".jpg") or filename.endswith(".png"): # Filter image files
        # Load the image
        image_path = os.path.join(input_directory, filename)
        image = Image.open(image_path)

        # Crop the image into pieces
        pieces = crop_image(image)

        # Save the pieces in the output directory
        for idx, piece in enumerate(pieces):
            piece_filename = f"{filename.split('.')[0]}_piece{idx}.jpg"
            piece_path = os.path.join(output_directory, piece_filename)
            piece.save(piece_path)

            # Display the positions of each piece
            print(f"Position of {piece_filename}: {idx % 2}, {idx // 2}")

print("Cropping completed.")


Position of WhatsApp Image 2024-03-27 at 13_piece0.jpg: 0, 0
Position of WhatsApp Image 2024-03-27 at 13_piece1.jpg: 1, 0
Position of WhatsApp Image 2024-03-27 at 13_piece2.jpg: 0, 1
Position of WhatsApp Image 2024-03-27 at 13_piece3.jpg: 1, 1
Cropping completed.


In [ ]:
!python /content/yolov5/detect.py  --weights /content/drive/MyDrive/ColabNotebooks/best.pt  --img 672 --conf 0.1 --source /content/drive/MyDrive/ColabNotebooks/test/imagesdecoupees

detect: weights=['/content/drive/MyDrive/ColabNotebooks/best.pt'], source=/content/drive/MyDrive/ColabNotebooks/test/imagesdecoupees, data=data/coco128.yaml, imgsz=[672, 672], conf_thres=0.1, iou_thres=0.45, max_det=1000, device=, view_img=False, save_txt=False, save_csv=False, save_conf=False, save_crop=False, nosave=False, classes=None, agnostic_nms=False, augment=False, visualize=False, update=False, project=runs/detect, name=exp, exist_ok=False, line_thickness=3, hide_labels=False, hide_conf=False, half=False, dnn=False, vid_stride=1
YOLOv5 🚀 v7.0-295-gac6c4383 Python-3.10.12 torch-2.2.1+cu121 CPU

Fusing layers... 
Model summary: 157 layers, 1760518 parameters, 0 gradients, 4.1 GFLOPs
image 1/4 /content/drive/MyDrive/ColabNotebooks/test/imagesdecoupees/WhatsApp Image 2024-03-27 at 13_piece0.jpg: 672x320 72 oranges, 125.5ms
image 2/4 /content/drive/MyDrive/ColabNotebooks/test/imagesdecoupees/WhatsApp Image 2024-03-27 at 13_piece1.jpg: 672x320 63 oranges, 128.0ms
image 3/4 /content/

In [ ]:
import os
from PIL import Image

# Function to reconstruct an image from pieces
def reconstruct_image(pieces, output_path):
    # Get dimensions of a piece
    piece_width, piece_height = pieces[0].size

    # Calculate dimensions of the reconstructed image
    image_width = piece_width * 2
    image_height = piece_height * 2

    # Create a new image to reconstruct the original image
    reconstructed_image = Image.new("RGB", (image_width, image_height))

    # Assemble the pieces to reconstruct the original image
    for idx, piece in enumerate(pieces):
        x = idx % 2
        y = idx // 2
        reconstructed_image.paste(piece, (x * piece_width, y * piece_height))

    # Save the reconstructed image
    reconstructed_image.save(output_path)

# Path to the directory containing image pieces
input_directory = "/content/yolov5/runs/detect/exp5"
output_directory = "/content/drive/MyDrive/ColabNotebooks/test/imagesoutput"

# Iterate through all image pieces in the directory
image_pieces = {}
for filename in sorted(os.listdir(input_directory)):  # Added sorted() here
    if filename.endswith(".jpg") or filename.endswith(".png"):  # Filter image files
        # Extract the name of the original image from the piece name
        image_name = filename.split("_")[0]

        # Load the image piece
        piece_path = os.path.join(input_directory, filename)
        piece = Image.open(piece_path)

        # Add the piece to the list of pieces associated with the original image
        if image_name not in image_pieces:
            image_pieces[image_name] = []
        image_pieces[image_name].append(piece)

# Before reconstituting each image, sort the pieces by their intended order
for image_name, pieces in image_pieces.items():
    # Assuming your pieces are named with an ending number indicating their order
    pieces_sorted = sorted(pieces, key=lambda x: int(x.filename.split('_')[-1].split('.')[0].replace('piece', '')))
    # Output path for the reconstructed image
    output_path = os.path.join(output_directory, f"{image_name}_reconstructed.jpg")
    # Reconstruct the image with the sorted pieces and save
    reconstruct_image(pieces_sorted, output_path)

print("Reconstruction completed.")


Reconstruction completed.


In [ ]:
import os
import shutil

def empty_directory(directory):
    """
    Vide un répertoire en supprimant tous ses fichiers et sous-répertoires.
    Args:
        directory (str): Chemin du répertoire à vider.
    """
    for filename in os.listdir(directory):
        file_path = os.path.join(directory, filename)
        try:
            if os.path.isfile(file_path) or os.path.islink(file_path):
                os.unlink(file_path)
            elif os.path.isdir(file_path):
                shutil.rmtree(file_path)
        except Exception as e:
            print('Erreur lors de la suppression du fichier %s : %s' % (file_path, e))

# Spécifiez les chemins des répertoires que vous souhaitez vider
directories_to_empty = [
    '/content/yolov5/runs/detect',
    '/content/drive/MyDrive/ColabNotebooks/test/video',
    '/content/drive/MyDrive/ColabNotebooks/test/imagesoutput',
    '/content/drive/MyDrive/ColabNotebooks/test/imagesinput',
    '/content/drive/MyDrive/ColabNotebooks/test/imagesdecoupees',

    # Ajoutez d'autres répertoires au besoin
]

# Videz chaque répertoire spécifié
for directory in directories_to_empty:
    print("Vidage du répertoire :", directory)
    empty_directory(directory)
